In [1]:
import pandas as pd
import requests
import schedule
import time
import os
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from dotenv import load_dotenv

In [ ]:
load_dotenv()  # Load environment variables from a .env file
API_KEY = os.getenv("CMC_API_KEY")  # Set in the environment variables
print(f"API Key: {API_KEY}") 
API_URL = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"

def fetch_crypto_data():
    """Fetches cryptocurrency data from CoinMarketCap API and saves it to a CSV file."""
    
    parameters = {
        'start': '1',
        'limit': '100',  # Fetch top 100 cryptocurrencies
        'convert': 'INR'  # Convert prices to Indian Rupees (INR)
    }
    
    headers = {
        'Accepts': 'application/json',
        'X-CMC_PRO_API_KEY': API_KEY,  # Secure API Key handling
    }
    
    try:
        response = requests.get(API_URL, headers=headers, params=parameters)
        response.raise_for_status()  # Raises an error for bad HTTP responses
        data = response.json()['data']
        
        # Normalize the nested JSON data
        df = pd.json_normalize(data)
        
        # Add timestamp
        df['timestamp'] = pd.Timestamp.now(tz='Asia/Kolkata')

        # Change Format
        df['timestamp'] = df['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
        
        # Define folder and file path
        folder_path = r"C:\Users\DELL\OneDrive\Desktop\DA Projects\Crypto-Market-Analysis-Dashboard\data"
        file_path = os.path.join(folder_path, "raw_data_2.csv")

        # Ensure the "data" folder exists
        os.makedirs(folder_path, exist_ok=True)

        # Save DataFrame to CSV (append if file exists)
        df.to_csv(file_path, index=False, mode='a', header=not os.path.isfile(file_path))

        print("Data fetched & saved successfully!")
        return df
    
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(f"API request failed: {e}")
        return None

# Schedule the data fetch job
# schedule.every(1).minutes.do(fetch_crypto_data) - for every minute
fetch_crypto_data()  # Fetch data immediately
schedule.every(1).hour.do(fetch_crypto_data) # for every hour

while True:
    schedule.run_pending()
    time.sleep(1)

In [2]:
# read crypto data
df = pd.read_csv("data/raw_data_2.csv")

# Data Cleaning

In [3]:
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', None) # Display all rows

In [4]:
df.columns

Index(['id', 'name', 'symbol', 'slug', 'num_market_pairs', 'date_added',
       'tags', 'max_supply', 'circulating_supply', 'total_supply',
       'infinite_supply', 'platform', 'cmc_rank',
       'self_reported_circulating_supply', 'self_reported_market_cap',
       'tvl_ratio', 'last_updated', 'quote.INR.price', 'quote.INR.volume_24h',
       'quote.INR.volume_change_24h', 'quote.INR.percent_change_1h',
       'quote.INR.percent_change_24h', 'quote.INR.percent_change_7d',
       'quote.INR.percent_change_30d', 'quote.INR.percent_change_60d',
       'quote.INR.percent_change_90d', 'quote.INR.market_cap',
       'quote.INR.market_cap_dominance', 'quote.INR.fully_diluted_market_cap',
       'quote.INR.tvl', 'quote.INR.last_updated', 'platform.id',
       'platform.name', 'platform.symbol', 'platform.slug',
       'platform.token_address', 'timestamp'],
      dtype='object')

In [5]:
df.shape

(300, 37)

In [6]:
df.head()

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,infinite_supply,platform,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,quote.INR.price,quote.INR.volume_24h,quote.INR.volume_change_24h,quote.INR.percent_change_1h,quote.INR.percent_change_24h,quote.INR.percent_change_7d,quote.INR.percent_change_30d,quote.INR.percent_change_60d,quote.INR.percent_change_90d,quote.INR.market_cap,quote.INR.market_cap_dominance,quote.INR.fully_diluted_market_cap,quote.INR.tvl,quote.INR.last_updated,platform.id,platform.name,platform.symbol,platform.slug,platform.token_address,timestamp
0,1,Bitcoin,BTC,bitcoin,11941,2010-07-13T00:00:00.000Z,"['mineable', 'pow', 'sha-256', 'store-of-value...",2.100000e+07,1.982510e+07,1.982510e+07,False,NaN,1,NaN,NaN,NaN,2025-02-16T12:21:00.000Z,8.428113e+06,1.376421e+12,-47.2158,-0.005551,-0.432512,0.555438,-5.181625,-7.541982,7.552868,1.670882e+14,59.6846,1.769904e+14,NaN,2025-02-16T12:21:04.000Z,NaN,NaN,NaN,NaN,NaN,2025-02-16 17:52:31
1,1027,Ethereum,ETH,ethereum,9931,2015-08-07T00:00:00.000Z,"['pos', 'smart-contracts', 'ethereum-ecosystem...",NaN,1.205527e+08,1.205527e+08,True,NaN,2,NaN,NaN,NaN,2025-02-16T12:20:00.000Z,2.349187e+05,8.921842e+11,-34.7556,0.151148,0.067744,1.885409,-20.788224,-30.216930,-11.541171,2.832008e+13,10.1153,2.832008e+13,NaN,2025-02-16T12:21:04.000Z,NaN,NaN,NaN,NaN,NaN,2025-02-16 17:52:31
2,52,XRP,XRP,xrp,1533,2013-08-04T00:00:00.000Z,"['medium-of-exchange', 'enterprise-solutions',...",1.000000e+11,5.781886e+10,9.998645e+10,False,NaN,3,NaN,NaN,NaN,2025-02-16T12:20:00.000Z,2.391597e+02,3.175126e+11,-44.3853,0.140255,-1.747728,12.792448,-15.685058,7.346096,146.346151,1.382794e+13,4.9390,2.391597e+13,NaN,2025-02-16T12:21:04.000Z,NaN,NaN,NaN,NaN,NaN,2025-02-16 17:52:31
3,825,Tether USDt,USDT,tether,116851,2015-02-25T00:00:00.000Z,"['stablecoin', 'asset-backed-stablecoin', 'eth...",NaN,1.419965e+11,1.435668e+11,True,NaN,4,NaN,NaN,NaN,2025-02-16T12:21:00.000Z,8.667781e+01,3.882885e+12,-39.7480,0.018114,-0.004756,-0.004713,0.021807,0.035361,-0.002118,1.230795e+13,4.3964,1.244406e+13,NaN,2025-02-16T12:21:04.000Z,1027.0,Ethereum,ETH,ethereum,0xdac17f958d2ee523a2206206994597c13d831ec7,2025-02-16 17:52:31
4,1839,BNB,BNB,bnb,2361,2017-07-25T00:00:00.000Z,"['marketplace', 'centralized-exchange', 'payme...",NaN,1.424776e+08,1.424776e+08,False,NaN,5,NaN,NaN,NaN,2025-02-16T12:21:00.000Z,5.792858e+04,1.552017e+11,-12.7128,0.305302,1.318623,7.320498,-8.143251,-7.022425,8.742494,8.253524e+12,2.9482,8.253524e+12,NaN,2025-02-16T12:21:04.000Z,NaN,NaN,NaN,NaN,NaN,2025-02-16 17:52:31


In [7]:
df.isnull().sum()  # Check for missing values

id                                      0
name                                    0
symbol                                  0
slug                                    0
num_market_pairs                        0
date_added                              0
tags                                    0
max_supply                            186
circulating_supply                      0
total_supply                            0
infinite_supply                         0
platform                              300
cmc_rank                                0
self_reported_circulating_supply      201
self_reported_market_cap              201
tvl_ratio                             279
last_updated                            0
quote.INR.price                         0
quote.INR.volume_24h                    0
quote.INR.volume_change_24h             0
quote.INR.percent_change_1h             0
quote.INR.percent_change_24h            0
quote.INR.percent_change_7d             0
quote.INR.percent_change_30d      

In [8]:
# Drop columns having more than 50% missing values
df_cleaned = df.copy()

df_cleaned.drop(columns=['platform', 'self_reported_circulating_supply', 'self_reported_market_cap', 'tvl_ratio', 'quote.INR.tvl'], inplace=True) 

In [9]:
df_cleaned['date_added'] = pd.to_datetime(df_cleaned['date_added'])
df_cleaned['last_updated'] = pd.to_datetime(df_cleaned['last_updated'])
df_cleaned['timestamp'] = pd.to_datetime(df_cleaned['timestamp'])

In [ ]:
df['max_supply'].apply(lambda x: f"{x:,.0f}" if not pd.isna(x) else "NaN")


In [16]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 32 columns):
 #   Column                              Non-Null Count  Dtype              
---  ------                              --------------  -----              
 0   id                                  300 non-null    int64              
 1   name                                300 non-null    object             
 2   symbol                              300 non-null    object             
 3   slug                                300 non-null    object             
 4   num_market_pairs                    300 non-null    int64              
 5   date_added                          300 non-null    datetime64[ns, UTC]
 6   tags                                300 non-null    object             
 7   max_supply                          114 non-null    float64            
 8   circulating_supply                  300 non-null    float64            
 9   total_supply                        300 non

In [ ]:
df_cleaned.dtypes

In [14]:
df_cleaned.head()

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,infinite_supply,cmc_rank,last_updated,quote.INR.price,quote.INR.volume_24h,quote.INR.volume_change_24h,quote.INR.percent_change_1h,quote.INR.percent_change_24h,quote.INR.percent_change_7d,quote.INR.percent_change_30d,quote.INR.percent_change_60d,quote.INR.percent_change_90d,quote.INR.market_cap,quote.INR.market_cap_dominance,quote.INR.fully_diluted_market_cap,quote.INR.last_updated,platform.id,platform.name,platform.symbol,platform.slug,platform.token_address,timestamp
0,1,Bitcoin,BTC,bitcoin,11941,2010-07-13 00:00:00+00:00,"['mineable', 'pow', 'sha-256', 'store-of-value...",2.100000e+07,1.982510e+07,1.982510e+07,False,1,2025-02-16 12:21:00+00:00,8.428113e+06,1.376421e+12,-47.2158,-0.005551,-0.432512,0.555438,-5.181625,-7.541982,7.552868,1.670882e+14,59.6846,1.769904e+14,2025-02-16T12:21:04.000Z,NaN,NaN,NaN,NaN,NaN,2025-02-16 17:52:31
1,1027,Ethereum,ETH,ethereum,9931,2015-08-07 00:00:00+00:00,"['pos', 'smart-contracts', 'ethereum-ecosystem...",NaN,1.205527e+08,1.205527e+08,True,2,2025-02-16 12:20:00+00:00,2.349187e+05,8.921842e+11,-34.7556,0.151148,0.067744,1.885409,-20.788224,-30.216930,-11.541171,2.832008e+13,10.1153,2.832008e+13,2025-02-16T12:21:04.000Z,NaN,NaN,NaN,NaN,NaN,2025-02-16 17:52:31
2,52,XRP,XRP,xrp,1533,2013-08-04 00:00:00+00:00,"['medium-of-exchange', 'enterprise-solutions',...",1.000000e+11,5.781886e+10,9.998645e+10,False,3,2025-02-16 12:20:00+00:00,2.391597e+02,3.175126e+11,-44.3853,0.140255,-1.747728,12.792448,-15.685058,7.346096,146.346151,1.382794e+13,4.9390,2.391597e+13,2025-02-16T12:21:04.000Z,NaN,NaN,NaN,NaN,NaN,2025-02-16 17:52:31
3,825,Tether USDt,USDT,tether,116851,2015-02-25 00:00:00+00:00,"['stablecoin', 'asset-backed-stablecoin', 'eth...",NaN,1.419965e+11,1.435668e+11,True,4,2025-02-16 12:21:00+00:00,8.667781e+01,3.882885e+12,-39.7480,0.018114,-0.004756,-0.004713,0.021807,0.035361,-0.002118,1.230795e+13,4.3964,1.244406e+13,2025-02-16T12:21:04.000Z,1027.0,Ethereum,ETH,ethereum,0xdac17f958d2ee523a2206206994597c13d831ec7,2025-02-16 17:52:31
4,1839,BNB,BNB,bnb,2361,2017-07-25 00:00:00+00:00,"['marketplace', 'centralized-exchange', 'payme...",NaN,1.424776e+08,1.424776e+08,False,5,2025-02-16 12:21:00+00:00,5.792858e+04,1.552017e+11,-12.7128,0.305302,1.318623,7.320498,-8.143251,-7.022425,8.742494,8.253524e+12,2.9482,8.253524e+12,2025-02-16T12:21:04.000Z,NaN,NaN,NaN,NaN,NaN,2025-02-16 17:52:31
